In [11]:
import json
import datetime
from math import log

In [12]:
# input file name: ./input/stockPrices_raw.json
# output file name: ./input/stockReturns.json
# json structure: crawl daily price data from yahoo finance
#          term (short/mid/long)
#         /         |         \
#   ticker A   ticker B   ticker C
#      /   \      /   \      /   \
#  date1 date2 date1 date2 date1 date2
#
# Note: short return: adjClose/open - 1
#       mid return: adjClose/adjClose(7 days ago) - 1
#       long return: adjClose/adjClose(28 days ago) - 1

In [23]:
def calc_mid_long_return(ticker, date, delta, priceSet):
    baseDate = datetime.datetime.strptime(date, "%Y-%m-%d")
    prevDate = (baseDate - datetime.timedelta(days=delta)).strftime("%Y-%m-%d")
    nextDate = (baseDate + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    if delta == 1:
        wkday = baseDate.weekday()
        if wkday == 0: # Monday
            prevDate = (baseDate - datetime.timedelta(days=3)).strftime("%Y-%m-%d")
        elif wkday == 4: # Friday
            nextDate = (baseDate + datetime.timedelta(days=3)).strftime("%Y-%m-%d")
        elif wkday == 5: # Saturday
            prevDate = (baseDate - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
            nextDate = (baseDate + datetime.timedelta(days=2)).strftime("%Y-%m-%d")
        elif wkday == 6: # Sunday
            prevDate = (baseDate - datetime.timedelta(days=2)).strftime("%Y-%m-%d")
            nextDate = (baseDate + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    try:
        if delta == 1:
            return_self = (priceSet[ticker]['adjClose'][date] - priceSet[ticker]['open'][date]) / priceSet[ticker]['open'][date]
            return_sp500 = (priceSet['^GSPC']['adjClose'][date] - priceSet['^GSPC']['open'][date]) / priceSet['^GSPC']['open'][date]
            return_self_per = round(return_self, 4) * 100
            return_sp500_per = round(return_sp500, 4) * 100
        
        else:
            return_self = (priceSet[ticker]['adjClose'][date] - priceSet[ticker]['adjClose'][prevDate]) / priceSet[ticker]['adjClose'][prevDate]
            return_sp500 = (priceSet['^GSPC']['adjClose'][date] - priceSet['^GSPC']['adjClose'][prevDate]) / priceSet['^GSPC']['adjClose'][prevDate]
            return_self_per = round(return_self, 4) * 100
            return_sp500_per = round(return_sp500, 4) * 100
            
        return True, round((return_self_per - return_sp500_per), 4) # relative return
    except:
        return False, 0

def main():
    raw_price_file = 'inputs/stockPrices_raw.json'
    with open(raw_price_file) as file:
        print("Loading price info ...")
        priceSet = json.load(file)
        dateSet = priceSet['AAPL']['adjClose'].keys()

    returns = {'short': {}, 'mid': {}, 'long': {}} # 1-depth dictionary
    for num, ticker in enumerate(priceSet):
        print(num, ticker)
        for term in ['short', 'mid', 'long']:
            returns[term][ticker] = {} # 2-depth dictionary
        for day in dateSet:
            date = datetime.datetime.strptime(day, "%Y-%m-%d").strftime("%Y%m%d") # change date 2014-01-01 to 20140101
            tag_short, return_short = calc_mid_long_return(ticker, day, 1, priceSet)
            tag_mid, return_mid = calc_mid_long_return(ticker, day, 7, priceSet)
            tag_long, return_long = calc_mid_long_return(ticker, day, 28, priceSet)
            if tag_short:
                returns['short'][ticker][date] = return_short
            if tag_mid:
                returns['mid'][ticker][date] = return_mid
            if tag_long:
                returns['long'][ticker][date] = return_long

    with open('./inputs/stockReturns.json', 'w') as outfile:
        json.dump(returns, outfile, indent=4)


In [24]:
if __name__ == "__main__":
    main()

Loading price info ...
0 ^GSPC
1 ADBE
2 GOOGL
3 AMZN
4 AMGN
5 AAPL
6 AVGO
7 CSCO
8 CMCSA
9 COST
10 FB
11 INTC
12 MSFT
13 NFLX
14 PYPL
15 PEP
16 SBUX
17 TXN
18 ABT
19 ACN
20 BABA
21 BUD
22 T
23 BAC
24 BBL
25 BA
26 BP
27 CVX
28 CHL
29 C
30 KO
31 XOM
32 FMX
33 HD
34 HON
35 HSBC
36 IBM
37 JPM
38 JNJ
39 LIN
40 MA
41 MCD
42 MDT
43 MRK
44 NKE
45 NVS
46 NVO
47 ORCL
48 PFE
49 PM
50 PG
51 RY
52 CRM
53 SAP
54 TSM
55 TMO
56 TD
57 TOT
58 TM
59 UN
60 UL
61 UNP
62 UTX
63 UNH
64 VZ
65 V
66 WMT
67 DIS
68 WFC
